In [1]:
import os
import pymorphy2

In [2]:
l = ["AND", "OR", "NOT"]

In [3]:
def to_normal_form(arr):
    morph = pymorphy2.MorphAnalyzer(lang='ru')

    normal_form = []

    for i, word in enumerate(arr):
        if word not in l:
            p = morph.parse(word)[0]
            word_in_normal_form = p.normal_form
            normal_form.append(word_in_normal_form)
        else:
            normal_form.append(word)

    return normal_form

In [4]:
def create_index(path_to_files):
    files = [f"{i}.txt" for i in range(100)]

    word_map_to_document = dict()

    for i, filename in enumerate(files):
        with open(f"{path_to_files}/{filename}", "r") as file:
            words = file.readline().split()
            for word in words:
                if word not in word_map_to_document:
                    word_map_to_document[word] = {i}
                else:
                    word_map_to_document.get(word).add(i)

    return word_map_to_document


In [5]:
def func_by_token(token, set_to_operate, set_to_insert):
    if token == "NOT":
        set_to_operate.difference_update(set_to_insert)

    if token == "AND":
        set_to_operate.intersection_update(set_to_insert)

    if token == "OR":
        set_to_operate.update(set_to_insert)

    return set_to_operate

In [6]:
def parse(list_of_tokens, dict_of_words, n=100):

    list_of_tokens = to_normal_form(list_of_tokens)

    if list_of_tokens[0] != "NOT":
        set_of_all_docs = set(dict_of_words[list_of_tokens[0]])
        i = 1
    else:
        set_of_all_docs = set(range(0, n))
        set_of_all_docs.difference_update(dict_of_words[list_of_tokens[1]])
        i = 2

    while i < len(list_of_tokens) - 1:
        token = list_of_tokens[i]
        next_token = list_of_tokens[i + 1]

        if next_token != "NOT":
            func_by_token(token, set_of_all_docs, dict_of_words[next_token])
            i += 2

        else:
            set_of_docs_without_next_token = set(range(0, n)).difference(dict_of_words[list_of_tokens[i + 2]])
            func_by_token(token, set_of_all_docs, set_of_docs_without_next_token)
            i += 3

    return set_of_all_docs


In [7]:
def simple_demo(search, n=6):
    dict_of_words = {"день": [0, 1, 2, 3],
                     "ночь": [1, 2, 4, 5],
                     "солнце": [2, 3, 4],
                     "море": [4]}

    list_of_tokens = search.split(" ")

    s = parse(list_of_tokens, dict_of_words, n)

    print(s)

In [8]:
simple_demo("NOT день AND ночь AND солнце")

{4}


In [9]:
simple_demo("день AND ночь OR море")

{1, 2, 4}


In [10]:
simple_demo("день AND ночь OR NOT море")

{0, 1, 2, 3, 5}


In [11]:
def main():
    dict_of_words = create_index("normal_form_docs")
    search1 = "школа AND инновация AND NOT медик"
    set1 = parse(search1.split(" "), dict_of_words)
    print(f"On search {search1} answear is {set1}")
    
    search2 = "научный AND библиотека AND медик"
    set2 = parse(search2.split(" "), dict_of_words)
    print(f"On search {search2} answear is {set2}")

In [12]:
main()

On search школа AND инновация AND NOT медик answear is {0, 15, 17, 20, 85, 22, 86, 24, 89, 87}
On search научный AND библиотека AND медик answear is {47}
